In [1]:
%reload_ext autoreload
%autoreload 2

from config import Config
from transcriptLoader import TranscriptData

# videoNames = [
#     "New Quizzes Video",
#     "Rearrange Playlist video",
#     "IMSE 514 Presentation",
#     "New Google Assignments in Canvas",
#     "Piazza Introduction Workshop",
# ]

config = Config()
config.videoToUse = "Piazza Introduction Workshop"
config.setFromEnv()


videoData = TranscriptData(config)

In [2]:
from topicExtractor import retrieveTopics

topicModeller = retrieveTopics(config, videoData)

Topic Model and Data loaded from saved files.


In [3]:
topicModeller.topicsOverTime

,Topic,Words,Frequency,Timestamp
0,0,"Piazza Platform Overview, , , ,",2,1899-12-31 23:59:57.897005000
1,1,"Piazza Classroom Management, , , ,",2,1899-12-31 23:59:57.897005000
2,-1,"Piazza Classroom Management, , , ,",2,1900-01-01 00:02:00.211590909
3,0,"Piazza Classroom Management, , , ,",2,1900-01-01 00:02:00.211590909
4,-1,"Educational platform features tutorial, , , ,",4,1900-01-01 00:03:59.893181818
5,-1,"Collaborative Educational Platform Features, ,...",2,1900-01-01 00:05:59.574772727
6,1,"Online Discussion Platform Features, , , ,",2,1900-01-01 00:05:59.574772727
7,-1,"Classroom Discussion Platform Features, , , ,",2,1900-01-01 00:07:59.256363636
8,1,"Anonymous Collaborative Editing, , , ,",2,1900-01-01 00:07:59.256363636
9,-1,"Educational Platform Interface Guide, , , ,",1,1900-01-01 00:09:58.937954545
